### 임베딩

### 학습 내용
 * 텍스트 임베딩(Text Embedding) 이해하기
 * 여러 입력에 대한 임베딩
 * 시멘틱 탐색
 * 코사인 유사도 예제

### 사전 준비
 * 구글 코랩 환경은 일정 시간이후에 초기화가 되기 때문에 두가지 작업을 매번 수행해야 함.
   * chatgpt.env 파일 생성이 필요.
     * 준비된 chatgpt.env를 내용을 변경하여 업로드 하거나 또는 API_KEY와 ORG_ID를 확인하여 생성한다.
   * pip install openai 설치

In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.4 MB/s eta 0:00:00


In [2]:
import matplotlib
import plotly
import scipy
import sklearn

### 텍스트 임베딩 이해하기

In [3]:
import os
import openai

def init_api():
    with open("chatgpt.env") as env:
        for line in env:
            key, value = line.strip().split("=")
            os.environ[key] = value

    openai.api_key = os.environ.get("API_KEY")
    openai.organization = os.environ.get("ORG_ID")

init_api()

response = openai.Embedding.create(
  model="text-embedding-ada-002",
  input="I am a programmer"
)

print(response)


{
  "object": "list",
  "data": [
    {
      "object": "embedding",
      "index": 0,
      "embedding": [
        -0.016830815002322197,
        -0.019729454070329666,
        -0.011307368986308575,
        -0.0164300799369812,
        -0.011127038858830929,
        0.02308226004242897,
        -0.025500020012259483,
        -0.00929702166467905,
        -0.010232066735625267,
        -0.024217672646045685,
        0.021091949194669724,
        0.0031591171864420176,
        0.005192840471863747,
        -0.01661708950996399,
        -0.00902318675071001,
        -0.012943698093295097,
        0.019943179562687874,
        -0.0009934855625033379,
        -0.009069939143955708,
        -0.0034897224977612495,
        0.0041876668110489845,
        0.015668686479330063,
        0.02130567468702793,
        -0.012222377583384514,
        -0.013043881393969059,
        0.006455151829868555,
        -0.0047119599767029285,
        -0.02023705095052719,
        0.005106015130877495,
      

In [12]:
print(response['data'][0]['embedding'])

[-0.016830815002322197, -0.019729454070329666, -0.011307368986308575, -0.0164300799369812, -0.011127038858830929, 0.02308226004242897, -0.025500020012259483, -0.00929702166467905, -0.010232066735625267, -0.024217672646045685, 0.021091949194669724, 0.0031591171864420176, 0.005192840471863747, -0.01661708950996399, -0.00902318675071001, -0.012943698093295097, 0.019943179562687874, -0.0009934855625033379, -0.009069939143955708, -0.0034897224977612495, 0.0041876668110489845, 0.015668686479330063, 0.02130567468702793, -0.012222377583384514, -0.013043881393969059, 0.006455151829868555, -0.0047119599767029285, -0.02023705095052719, 0.005106015130877495, -0.015134375542402267, 0.03371506184339523, -0.00553012453019619, 0.002330934163182974, -0.019609235227108, -0.014266119338572025, -0.01651022769510746, -0.0017999621341004968, 0.011374157853424549, 0.0017815951723605394, -0.002770071616396308, 0.02986801601946354, 0.029493998736143112, -0.008969755843281746, -0.000998494797386229, -0.01489393

#### 여러 입력에 대한 임베딩

In [13]:
import os
import openai

def init_api():
    with open("chatgpt.env") as env:
        for line in env:
            key, value = line.strip().split("=")
            os.environ[key] = value

    openai.api_key = os.environ.get("API_KEY")
    openai.organization = os.environ.get("ORG_ID")

init_api()

response = openai.Embedding.create(
	model="text-embedding-ada-002",
	input=["I am a programmer", "I am a writer"]
)

for data in response["data"]:
  print(data["embedding"])


[-0.01685187593102455, -0.019698387011885643, -0.011305858381092548, -0.016437595710158348, -0.011158855631947517, 0.023159636184573174, -0.025538410991430283, -0.009294591844081879, -0.010189972817897797, -0.024202020838856697, 0.021141687408089638, 0.003180607920512557, 0.005205238703638315, -0.016597962006926537, -0.00902063213288784, -0.012949618510901928, 0.019938938319683075, -0.0010966749396175146, -0.009054041467607021, -0.0035046825651079416, 0.004226332996040583, 0.015689218416810036, 0.021302055567502975, -0.012167830020189285, -0.013029801659286022, 0.006534946616739035, -0.0047876168973743916, -0.02024630643427372, 0.005078281741589308, -0.015181389637291431, 0.0337839350104332, -0.005515949334949255, 0.0023821149952709675, -0.019591476768255234, -0.01432610023766756, -0.01657123491168022, -0.0017590231727808714, 0.011325904168188572, 0.0017573527293279767, -0.0028064188081771135, 0.029908405616879463, 0.02942730486392975, -0.008940448053181171, -0.001001457218080759, -0.0

### 시맨틱 검색(Semantic Search) 탐색

#### 인증

In [14]:
import openai
import os
import pandas as pd
import numpy as np

def init_api():
    with open("chatgpt.env") as env:
        for line in env:
            key, value = line.strip().split("=")
            os.environ[key] = value

    openai.api_key = os.environ.get("API_KEY")
    openai.organization = os.environ.get("ORG_ID")

init_api()

#### words.csv를 만들고, 이를 불러오기
 * words.csv파일의 내용은 책을 보고 생성을 해 주세요.구글 코랩에 업로드를 하거나 직접 words.csv 파일을 생성합니다.

In [15]:
df = pd.read_csv('words.csv')
print(df)

          text
0        apple
1       banana
2       cherry
3          dog
4          cat
5        house
6          car
7         tree
8        phone
9     computer
10  television
11        book
12       music
13        food
14       water
15         sky
16         air
17         sun
18        moon
19        star
20       ocean
21        desk
22         bed
23        sofa
24        lamp
25      carpet
26      window
27        door
28       floor
29     ceiling
30        wall
31       clock
32       watch
33     jewelry
34        ring
35    necklace
36    bracelet
37     earring
38      wallet
39         key
40       photo


In [16]:
# 임베딩 함수 불러오기
from openai.embeddings_utils import get_embedding

# 데이터 프레임의 각 단어에 대한 임베딩 가져오기
df['embedding'] = df['text'].apply(lambda x: get_embedding(x, engine='text-embedding-ada-002'))

# 정보 확인
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       41 non-null     object
 1   embedding  41 non-null     object
dtypes: object(2)
memory usage: 784.0+ bytes
None


In [17]:
df

,text,embedding
0,apple,"[0.007793936878442764, -0.023018505424261093, ..."
1,banana,"[-0.013941187411546707, -0.032889608293771744,..."
2,cherry,"[0.0064801559783518314, -0.018939442932605743,..."
3,dog,"[-0.003389080287888646, -0.01774153858423233, ..."
4,cat,"[-0.007023478392511606, -0.01733333431184292, ..."
5,house,"[-0.007149112410843372, 0.007184312213212252, ..."
6,car,"[-0.007446334231644869, -0.021665772423148155,..."
7,tree,"[-0.004675603471696377, -0.013128847815096378,..."
8,phone,"[-0.0014481429243460298, -0.022867493331432343..."
9,computer,"[-0.003195784520357847, -0.01424823421984911, ..."


In [18]:
# df의 내용을 csv파일로 저장하기
df.to_csv("embeddings.csv")

#### 시멘틱 탐색 전체 코드

In [22]:
import openai
import os
import pandas as pd
import numpy as np
from openai.embeddings_utils import get_embedding
from openai.embeddings_utils import cosine_similarity

def init_api():
    with open("chatgpt.env") as env:
        for line in env:
            key, value = line.strip().split("=")
            os.environ[key] = value

    openai.api_key = os.environ.get("API_KEY")
    openai.organization = os.environ.get("ORG_ID")

init_api()

# words.csv is a csv file with a column named 'text' containing words
# words.csv 파일 불러오기
df = pd.read_csv('words.csv')

# text열의 각 행에 대해 임베딩을 생성하여 ‘embedding’열로 추가
df['embedding'] = df['text'].apply(lambda x: get_embedding(x, engine='text-embedding-ada-002'))

# df의 데이터 프레임 객체를 csv 파일로 생성
df.to_csv('embeddings.csv')

# csv 파일 읽기
df = pd.read_csv('embeddings.csv')

# embedding 열의 값을 문자열에서 Numpy 배열로 변환하여 저장
df['embedding'] = df['embedding'].apply(eval).apply(np.array)

# 사용자로부터 검색 용어를 입력 받는다.
user_search = input('Enter a search term: ')

# 검색 용어에 대한 임베딩을 생성
search_term_embedding = get_embedding(user_search, engine='text-embedding-ada-002')

# ‘embedding’의 열의 각 행에 대한 코사인 유사도를 계산하여 ‘similarity’열로 추가
df['similarity'] = df['embedding'].apply(lambda x: cosine_similarity(x, search_term_embedding))

print(df)


Enter a search term: office
    Unnamed: 0        text                                          embedding  \
0            0       apple  [0.007745007518678904, -0.023043110966682434, ...   
1            1      banana  [-0.013941187411546707, -0.032889608293771744,...   
2            2      cherry  [0.006536407396197319, -0.0189522635191679, -0...   
3            3         dog  [-0.0033466550521552563, -0.01776769571006298,...   
4            4         cat  [-0.007023478392511606, -0.01733333431184292, ...   
5            5       house  [-0.007149112410843372, 0.007184312213212252, ...   
6            6         car  [-0.007446334231644869, -0.021665772423148155,...   
7            7        tree  [-0.004675603471696377, -0.013128847815096378,...   
8            8       phone  [-0.0014547440223395824, -0.02284170128405094,...   
9            9    computer  [-0.0031430479139089584, -0.014253114350140095...   
10          10  television  [-0.004814683459699154, -0.01997491344809532, ...   


In [23]:
df.columns

Index(['Unnamed: 0', 'text', 'embedding', 'similarity'], dtype='object')

In [24]:
df

,Unnamed: 0,text,embedding,similarity
0,0,apple,"[0.007745007518678904, -0.023043110966682434, ...",0.830553
1,1,banana,"[-0.013941187411546707, -0.032889608293771744,...",0.805884
2,2,cherry,"[0.006536407396197319, -0.0189522635191679, -0...",0.792315
3,3,dog,"[-0.0033466550521552563, -0.01776769571006298,...",0.828753
4,4,cat,"[-0.007023478392511606, -0.01733333431184292, ...",0.802086
5,5,house,"[-0.007149112410843372, 0.007184312213212252, ...",0.874442
6,6,car,"[-0.007446334231644869, -0.021665772423148155,...",0.821615
7,7,tree,"[-0.004675603471696377, -0.013128847815096378,...",0.825798
8,8,phone,"[-0.0014547440223395824, -0.02284170128405094,...",0.853305
9,9,computer,"[-0.0031430479139089584, -0.014253114350140095...",0.861793


#### 검색 단어에 대한 상위 유사도 10개 출력하기

In [27]:
# 필수 라이브러리 불러오기
import openai
import os
import pandas as pd
import numpy as np

from openai.embeddings_utils import get_embedding
from openai.embeddings_utils import cosine_similarity

# open API 사용을 위한 인증
def init_api():
    with open("chatgpt.env") as env:
        for line in env:
            key, value = line.strip().split("=")
            os.environ[key] = value

    openai.api_key = os.environ.get("API_KEY")
    openai.organization = os.environ.get("ORG_ID")

init_api()

# words.csv 파일 불러오기
df = pd.read_csv('words.csv')

# text열의 각 행의 단어에 대해 임베딩을 생성하여 ‘embedding’열로 추가
df['embedding'] = df['text'].apply(lambda x: get_embedding(x, engine='text-embedding-ada-002'))

# df의 데이터 프레임 객체를 csv 파일로 생성
df.to_csv('embeddings.csv')

# 생성된 임베딩 파일을 다시 읽어오기. csv 파일 읽기
df = pd.read_csv('embeddings.csv')

# embedding 열의 값을 문자열에서 Numpy 배열로 변환하여 저장
df['embedding'] = df['embedding'].apply(eval).apply(np.array)

# 사용자로부터 검색 용어를 입력 받는다.
user_search = input('Enter a search term: ')

# 검색 용어에 대한 임베딩을 생성한다.
search_term_embedding = get_embedding(user_search, engine='text-embedding-ada-002')

# ‘embedding’의 열의 각 행에 대한 코사인 유사도를 계산하여 ‘similarity’열로 추가
df['similarity'] = df['embedding'].apply(lambda x: cosine_similarity(x, search_term_embedding))

# 'similarity'을 기준으로 데이터 프레임 정렬
df = df.sort_values(by='similarity', ascending=False)

# 유사도 10개 확인
print(df.head(10))


Enter a search term: office
    Unnamed: 0      text                                          embedding  \
21          21      desk  [0.01280702743679285, -0.02078762836754322, -0...   
5            5     house  [-0.007149112410843372, 0.007184312213212252, ...   
9            9  computer  [-0.0031430479139089584, -0.014253114350140095...   
28          28     floor  [0.018728043884038925, -0.02113967388868332, -...   
8            8     phone  [-0.0014547440223395824, -0.02284170128405094,...   
11          11      book  [-0.0068463170900940895, -0.01912514865398407,...   
24          24      lamp  [0.006757527589797974, -0.00874999724328518, 0...   
27          27      door  [-0.004880927968770266, -0.02699458971619606, ...   
40          40     photo  [0.004259202629327774, -0.031308699399232864, ...   
13          13      food  [0.022216901183128357, -0.026853354647755623, ...   

    similarity  
21    0.890088  
5     0.874442  
9     0.861793  
28    0.861533  
8     0.853305  


In [26]:
df.head(10)

,Unnamed: 0,text,embedding,similarity
21,21,desk,"[0.01280702743679285, -0.02078762836754322, -0...",0.818071
8,8,phone,"[-0.0014547440223395824, -0.02284170128405094,...",0.801991
9,9,computer,"[-0.0031430479139089584, -0.014253114350140095...",0.796250
28,28,floor,"[0.0187282245606184, -0.021138105541467667, -0...",0.783616
27,27,door,"[-0.004880927968770266, -0.02699458971619606, ...",0.782248
23,23,sofa,"[0.011774417944252491, -0.011563675478100777, ...",0.780940
24,24,lamp,"[0.0068565718829631805, -0.008870316669344902,...",0.778641
40,40,photo,"[0.004259202629327774, -0.031308699399232864, ...",0.777658
3,3,dog,"[-0.003353922860696912, -0.01773897558450699, ...",0.777265
5,5,house,"[-0.007149112410843372, 0.007184312213212252, ...",0.776420


#### 시멘틱 탐색 - 한글 단어 입력 후, 코사인 유사도 확인해 보기

In [ ]:
# 필수 라이브러리 불러오기
import openai
import os
import pandas as pd
import numpy as np

from openai.embeddings_utils import get_embedding
from openai.embeddings_utils import cosine_similarity

# open API 사용을 위한 인증
def init_api():
    with open("chatgpt.env") as env:
        for line in env:
            key, value = line.strip().split("=")
            os.environ[key] = value

    openai.api_key = os.environ.get("API_KEY")
    openai.organization = os.environ.get("ORG_ID")

init_api()

# words.csv 파일 불러오기
df = pd.read_csv('words.csv')

# text열의 각 행의 단어에 대해 임베딩을 생성하여 ‘embedding’열로 추가
df['embedding'] = df['text'].apply(lambda x: get_embedding(x, engine='text-embedding-ada-002'))

# df의 데이터 프레임 객체를 csv 파일로 생성
df.to_csv('embeddings.csv')

# 생성된 임베딩 파일을 다시 읽어오기. csv 파일 읽기
df = pd.read_csv('embeddings.csv')

# embedding 열의 값을 문자열에서 Numpy 배열로 변환하여 저장
df['embedding'] = df['embedding'].apply(eval).apply(np.array)

# 사용자로부터 검색 용어를 입력 받는다.
user_search = input('Enter a search term: ')

# 검색 용어에 대한 임베딩을 생성한다.
search_term_embedding = get_embedding(user_search, engine='text-embedding-ada-002')

# ‘embedding’의 열의 각 행에 대한 코사인 유사도를 계산하여 ‘similarity’열로 추가
df['similarity'] = df['embedding'].apply(lambda x: cosine_similarity(x, search_term_embedding))

# 'similarity'을 기준으로 데이터 프레임 정렬
df = df.sort_values(by='similarity', ascending=False)

# 유사도 10개 확인
print(df.head(10))


Enter a search term: 사무실
    Unnamed: 0      text                                          embedding  \
21          21      desk  [0.01280386932194233, -0.02075531706213951, -0...   
8            8     phone  [-0.0014547440223395824, -0.02284170128405094,...   
9            9  computer  [-0.0031430479139089584, -0.014253114350140095...   
28          28     floor  [0.018672894686460495, -0.02109830267727375, -...   
27          27      door  [-0.004880927968770266, -0.02699458971619606, ...   
23          23      sofa  [0.011774417944252491, -0.011563675478100777, ...   
24          24      lamp  [0.006757527589797974, -0.00874999724328518, 0...   
40          40     photo  [0.004259202629327774, -0.031308699399232864, ...   
3            3       dog  [-0.0033466550521552563, -0.01776769571006298,...   
5            5     house  [-0.00718662841245532, 0.007179589476436377, -...   

    similarity  
21    0.818127  
8     0.801991  
9     0.796250  
28    0.783703  
27    0.782248  
23 

### 코사인 유사도 예제

#### numpy 라이브러리 이용한 코사인 유사도 예제

In [28]:
# numpy와 numpy.linalg에서 norm을 가져옵니다.
import numpy as np
from numpy.linalg import norm

# 두 벡터를 정의합니다.
A = np.array([2,3,5,2,6,7,9,2,3,4])
B = np.array([3,6,3,1,0,9,2,3,4,5])

# 벡터들을 출력합니다.
print("Vector A: {}".format(A))
print("Vector B: {}".format(B))

# 코사인 유사도를 계산합니다.
cosine = np.dot(A,B)/(norm(A)*norm(B))

# 코사인 유사도를 출력합니다.
print("Cosine Similarity between A and B: {}".format(cosine))


Vector A: [2 3 5 2 6 7 9 2 3 4]
Vector B: [3 6 3 1 0 9 2 3 4 5]
Cosine Similarity between A and B: 0.7539959431593041


#### Scipy 라이브러리를 활용한 예제

In [29]:
# 필요 라이브러리를 불러옵니다.
import numpy as np
from scipy import spatial

# 두 벡터들을 정의합니다.
A = np.array([2,3,5,2,6,7,9,2,3,4])
B = np.array([3,6,3,1,0,9,2,3,4,5])

# 벡터들을 출력합니다.
print("Vector A: {}".format(A))
print("Vector B: {}".format(B))

# 코사인 유사도를 계산합니다.
cosine = 1 - spatial.distance.cosine(A, B)

# 코사인 유사도를 출력합니다.
print("Cosine Similarity between A and B: {}".format(cosine))


Vector A: [2 3 5 2 6 7 9 2 3 4]
Vector B: [3 6 3 1 0 9 2 3 4 5]
Cosine Similarity between A and B: 0.753995943159304


#### 사이킷 런을 이용한 코사인 유사도 출력

In [30]:
# 필요 라이브러리를 불러옵니다.
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 두 벡터를 정의합니다.
A = np.array([2,3,5,2,6,7,9,2,3,4])
B = np.array([3,6,3,1,0,9,2,3,4,5])

# 벡터들을 출력합니다.
print("Vector A: {}".format(A))
print("Vector B: {}".format(B))

# 코사인 유사도를 계산합니다.
cosine = cosine_similarity([A],[B])

# 코사인 유사도를 출력합니다.
print("Cosine Similarity: {}".format(cosine[0][0]))


Vector A: [2 3 5 2 6 7 9 2 3 4]
Vector B: [3 6 3 1 0 9 2 3 4 5]
Cosine Similarity: 0.7539959431593041
